In [2]:
#from nytimesarticle import articleAPI
#api = articleAPI('a0HA3uBISDkGyvUGR3FeoAGybtDVPPM5')

In [4]:
#articles = api.search( q = 'immigration', 
#     begin_date = 20120101, end_date =20120201 )

# create TIDF on an example call from R

https://towardsdatascience.com/latent-semantic-analysis-sentiment-classification-with-python-5f657346f6a3